В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [132]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [133]:
X_train = np.zeros(34); X_train[0:10] = 1;
X_test = np.zeros(16); X_test[0:4] = 1;
print X_train
print X_test


[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [134]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [135]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [136]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [137]:
print "95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(X_train, X_test)

95% confidence interval for a difference between proportions: [-0.217558, 0.305793]


In [138]:
print proportions_diff_z_test(proportions_diff_z_stat_ind(X_train, X_test), alternative='greater')

0.372930458725


In [139]:
import sklearn
from sklearn import cross_validation, datasets, linear_model, metrics,ensemble

In [140]:
data = pd.read_csv("banknotes.txt",delimiter='\t')
data.head()
y = data.real
X = data.drop('real', axis=1)

In [141]:
X_train, X_test, y_train, y_test  = cross_validation.train_test_split(X, y, test_size=50, random_state=1)

In [142]:
X_train.head()

,X1,X2,X3,X4,X5,X6
98,215.1,130.0,129.8,9.1,10.2,141.5
123,215.1,130.7,130.4,10.5,11.2,139.7
119,214.4,130.2,129.9,10.1,12.0,139.2
53,215.4,130.2,130.2,7.6,10.9,141.6
33,215.6,130.4,130.1,8.4,10.3,141.0


In [143]:
X_train_123 = X_train.drop(['X4','X5','X6'],axis=1)
X_train_456 = X_train.drop(['X1','X2','X3'], axis=1)
X_test_123 = X_test.drop(['X4','X5','X6'],axis=1)
X_test_456 = X_test.drop(['X1','X2','X3'],axis=1)

In [144]:
linear_regressor = linear_model.LogisticRegression()
linear_regressor.fit(X_train_123,y_train)
linear_predict = linear_regressor.predict(X_test_123)
error = metrics.mean_absolute_error(y_test, linear_predict)
lin_deviation_123 = map(lambda x, y: abs(x-y), linear_predict, y_test)
print 'error - ', error

error -  0.2


In [145]:
linear_regressor = linear_model.LogisticRegression()
linear_regressor.fit(X_train_456,y_train)
linear_predict = linear_regressor.predict(X_test_456)
error = metrics.mean_absolute_error(y_test, linear_predict)
lin_deviation_456 = map(lambda x, y: abs(x-y), linear_predict, y_test)
print 'error - ', error

error -  0.02


In [146]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [150]:
print stats.ttest_rel(pd.DataFrame(lin_deviation_123), 
                    pd.DataFrame(lin_deviation_456))

print "95%% confidence interval for a difference between proportions: [%f, %f]" \
      % proportions_diff_confint_rel(lin_deviation_123, lin_deviation_456)


Ttest_relResult(statistic=array([ 2.90906971]), pvalue=array([ 0.00543628]))
95% confidence interval for a difference between proportions: [0.059945, 0.300055]


In [162]:
m1 = 525
s1 = 100
n1 = 200000

m2 = 541.4
n2 = 100

t = (m2-m1)/(100/(100**0.5))              
a = 1-stats.norm.cdf(t)


In [163]:
print t, a

1.64 0.0505025834741


In [167]:
a = stats.norm.cdf(t)
print a

0.949497416526
